In [43]:
import sqlite3
import pandas as pd
from bs4 import BeautifulSoup
from tqdm import tqdm
import re
from sklearn.feature_extraction.text import CountVectorizer 
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from gensim.models import Word2Vec
import numpy as np

In [44]:
conn = sqlite3.connect("database.sqlite")

In [45]:
data_set = pd.read_sql_query("SELECT * FROM Reviews WHERE Score != 3 LIMIT  5000",conn)

In [46]:
data_set.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [47]:
#### converting score column from 1,2,3,4 to binary  ....1 for 3 and 4 0 for 1 and 2

In [48]:
def participation(x):
    if (x>3):
        return 1
    elif (x<3):
        return 0

In [49]:
Score2 = list(map(participation,data_set["Score"]))

In [50]:
#### print(Score2)

In [51]:
data_set["Score"] = Score2 

In [52]:
### data_set.head(5)

In [53]:
#### now let us perform certain data reforming line 

In [54]:
data_set.shape

(5000, 10)

In [55]:
#### performing data deplications

In [56]:
sorted_data = data_set.sort_values( 'ProductId' , inplace = False , axis = 0 , ascending = True , kind = "quicksort" , na_position = "last")
final = sorted_data.drop_duplicates( subset = {"UserId" , "ProfileName" , "Time" ,"Text"} , inplace = False , keep = "first")

In [57]:
final.shape

(4986, 10)

In [58]:
#### those data sets for which helpfullness numerator is greater than helpfullness denominator

In [59]:
final = final[final["HelpfulnessNumerator"]<=final["HelpfulnessDenominator"]]

In [60]:
final.shape

(4986, 10)

In [61]:
final["Text"].values[0]

'Why is this $[...] when the same product is available for $[...] here?<br />http://www.amazon.com/VICTOR-FLY-MAGNET-BAIT-REFILL/dp/B00004RBDY<br /><br />The Victor M380 and M502 traps are unreal, of course -- total fly genocide. Pretty stinky, but only right nearby.'

In [62]:
### as we can see that there is a lot of junks in each text which has to be removed before converting it into vectors

In [63]:
##3 now it is time to use stopwords and portstemmer

In [64]:
stop_words = set(stopwords.words('english'))

In [65]:
stop_words.add("br")    ### as when html tags like < br > </ br > are removed br is not eleminated

In [66]:
print(stop_words)

{'it', 'couldn', 'below', "wasn't", 'while', 'weren', "hasn't", 'their', 'off', 'will', "haven't", 'up', 'being', 'the', 'didn', "you've", 'having', 'at', 'her', 'itself', 'were', "you'd", 'through', 'not', 'because', 'yours', 'about', 'own', 'had', 'been', 'again', 'aren', 'am', 'most', 'me', 'during', 'or', 'some', 'what', 'nor', 'herself', 'on', 'of', 'above', 'should', "mustn't", 've', 'yourself', 'you', 'only', 'ourselves', 'hadn', 'down', 'y', "hadn't", 'isn', 'no', 'further', 're', 'ain', 'hasn', 'out', 'him', 'against', 'd', 'who', 'o', 'was', 's', 'so', 'needn', 'too', 'which', 'theirs', 'our', 'did', 'he', 'is', 'until', 'now', 'does', 'but', 't', 'these', 'shan', 'your', 'doesn', 'once', "didn't", 'those', 'we', 'into', "aren't", 'before', 'and', 'as', 'in', 'same', "needn't", 'this', "mightn't", 'a', 'all', 'hers', 'yourselves', 'with', 'each', "she's", 'then', 'himself', 'them', 'that', 'be', 'other', 'my', 'are', 'when', 'themselves', 'mightn', 'i', 'over', 'where', 'wasn

In [67]:
type(stop_words)

set

In [69]:
stop_words.remove('not')
stop_words.remove('nor')
stop_words.remove('down')
stop_words.remove('no')

KeyError: 'not'

In [70]:
stem  = nltk.stem.SnowballStemmer("english")

In [71]:
stem.stem("tasty")

'tasti'

In [72]:
### using BeautifulSoup for removing html tags

In [73]:
def denocrate(phase):
    phase = re.sub(r"\'t"," not ",phase)
    phase = re.sub(r"\'ll", " will ",phase)
    phase = re.sub(r"\'re" , " are " , phase)
    phase = re.sub(r"\'d" , " would " , phase)
    phase = re.sub(r"\'m" , " am " , phase)
    phase = re.sub(r"\'ve" , " have " , phase)
    phase = re.sub(r"\'s" , " is " , phase)
    return phase

In [74]:
texts = []
for text in tqdm(final["Text"]):
    soup = BeautifulSoup(text , "lxml")        ###### remove html tags
    text = soup.get_text()
    text = re.sub(r"http\S+", "", text)           #### remove links 
    text = denocrate(text)
    text = re.sub("\S*\d\S*", "", text).strip()     #### removes wods which have deciml numbers
    text = re.sub('[^A-Za-z]+', " ", text).strip()   ####  removes special characters
    text = text.lower()
    text = ' '.join(word.lower() for word in text.split() if word.lower() not in stop_words)    ## these two lines which removs 
    ##text = ' '.join(stem.stem(word) for word in text.split() if len(word)>2)                stopwords and stemmerise the words are removed because of tfidf and w2v
    text = ' '.join(word for word in text.split() if len(word)>2)
    texts.append(text)

100%|█████████████████████████████████████████████████████████████████████████████| 4986/4986 [00:06<00:00, 802.39it/s]


In [75]:
texts[0]

'product available victor traps unreal course total fly genocide pretty stinky right nearby'

In [76]:
final["Cleaned_text"] = texts

In [77]:
final.shape

(4986, 11)

In [78]:
### cleaning summary of reviews

In [79]:
summary = []
for text in tqdm(final["Summary"]):
    soup = BeautifulSoup(text , "lxml")        ###### remove html tags
    text = soup.get_text()
    text = re.sub(r"http\S+", "", text)           #### remove links 
    text = denocrate(text)
    text = re.sub("\S*\d\S*", "", text).strip()     #### removes wods which have deciml numbers
    text = re.sub('[^A-Za-z]+', " ", text).strip()   ####  removes special characters
   ## text = ' '.join(word.lower() for word in text.split() if word.lower() not in stop_words)
    ## text = ' '.join(stem.stem(word) for word in text.split() if len(word)>2)
    summary.append(text)

100%|████████████████████████████████████████████████████████████████████████████| 4986/4986 [00:04<00:00, 1128.35it/s]


In [80]:
final["Cleaned_Summary"] = summary

In [81]:
final.shape

(4986, 12)

In [82]:
final["Cleaned_text"][0]

'bought several vitality canned dog food products found good quality product looks like stew processed meat smells better labrador finicky appreciates product better'

In [83]:
### now we have a problem that each time we want to use these vectors we have to run ths whole file so we either we could store
### the vector converted form of these cleaned texts to excel or database but this is too time consuming so i will store this 
##3 this cleaned text to new database

In [84]:
conn2 = sqlite3.connect("final_2.sqlite")

In [85]:
cursor_object = conn2.cursor()
conn2.text_factory = str
final.to_sql('Reviews', conn2 ,schema = None ,if_exists='replace' ,index = False )

In [45]:
### now it is time for featurisation(vectorisation of texts)

In [46]:
### BOW

In [47]:
count = CountVectorizer()
count.fit(final["Cleaned_text"])
bow_vector = count.transform(final["Cleaned_text"])
bow_vector.shape

(4986, 13237)

In [48]:
type(bow_vector)

scipy.sparse.csr.csr_matrix

In [49]:
### trying to store BOW vector into a new excel file
bow_vector2 = bow_vector.toarray()
bow_df = pd.DataFrame(data=bow_vector2)
bow_df.columns = np.arange(13237)
bow_df.index = np.arange(4986)

In [50]:
bow_df["Score"] = final["Score"]

In [51]:
print(bow_df.head(5))

   0  1  2  3  4  5  6  7  8  9  ...  13228  13229  13230  13231  13232  \
0  0  0  0  0  0  0  0  0  0  0  ...      0      0      0      0      0   
1  0  0  0  0  0  0  0  0  0  0  ...      0      0      0      0      0   
2  0  0  0  0  0  0  0  0  0  0  ...      0      0      0      0      0   
3  0  0  0  0  0  0  0  0  0  0  ...      0      0      0      0      0   
4  0  0  0  0  0  0  0  0  0  0  ...      0      0      0      0      0   

   13233  13234  13235  13236  Score  
0      0      0      0      0    1.0  
1      0      0      0      0    0.0  
2      0      0      0      0    1.0  
3      0      0      0      0    0.0  
4      0      0      0      0    1.0  

[5 rows x 13238 columns]


In [52]:
###  writer = pd.ExcelWriter("bow.xlsx")
###  bow_df.to_excel(writer )dd

In [53]:
### BI-GRAM and TRI-GRAM

In [54]:
count = CountVectorizer(ngram_range= (1,2) )
count.fit(final["Cleaned_text"])
bigram_vector = count.transform(final["Cleaned_text"])
bigram_vector.shape

(4986, 144190)

In [55]:
print(type(bow_vector))
print(type(bigram_vector))

<class 'scipy.sparse.csr.csr_matrix'>
<class 'scipy.sparse.csr.csr_matrix'>


In [56]:
print(bow_vector[0])
print(bigram_vector[0])

  (0, 425)	1
  (0, 570)	1
  (0, 744)	1
  (0, 1524)	1
  (0, 2653)	1
  (0, 4538)	1
  (0, 4582)	1
  (0, 4849)	1
  (0, 5445)	1
  (0, 6074)	2
  (0, 7616)	1
  (0, 7885)	1
  (0, 7951)	1
  (0, 8895)	1
  (0, 8977)	1
  (0, 9785)	1
  (0, 10021)	1
  (0, 11123)	1
  (0, 11766)	1
  (0, 11836)	1
  (0, 12020)	1
  (0, 12090)	1
  (0, 12397)	1
  (0, 12620)	1
  (0, 12900)	1
  (0, 12946)	1
  (0, 5084)	1
  (0, 7124)	1
  (0, 8494)	1
  (0, 9230)	1
  (0, 11100)	1
  (0, 11119)	1
  (0, 19357)	1
  (0, 19646)	1
  (0, 29424)	1
  (0, 29463)	1
  (0, 44987)	1
  (0, 44989)	1
  (0, 45587)	1
  (0, 45949)	1
  (0, 49024)	1
  (0, 49025)	1
  (0, 56459)	1
  (0, 56538)	1
  (0, 61868)	2
  (0, 61963)	1
  (0, 63236)	1
  (0, 79464)	1
  (0, 83146)	1
  (0, 83432)	1
  (0, 86486)	1
  :	:
  (0, 95865)	1
  (0, 96713)	1
  (0, 96849)	1
  (0, 102589)	1
  (0, 102645)	1
  (0, 104066)	1
  (0, 104161)	1
  (0, 114080)	1
  (0, 114082)	1
  (0, 120579)	1
  (0, 122782)	1
  (0, 125907)	1
  (0, 126658)	1
  (0, 130004)	1
  (0, 130013)	1
  (0, 130375)	1

In [57]:
#### tfidf_vectorizer

In [58]:
tfidf  = TfidfVectorizer(ngram_range = (1,2) , min_df = 2)
tfidf_model = tfidf.fit(final["Cleaned_text"])
print("some feature names of the tfidf are" ,tfidf_model.get_feature_names()[0:10])

tfidf_vector = tfidf_model.transform(final["Cleaned_text"])
print("the shape of tfidf vector is", tfidf_vector.shape)

some feature names of the tfidf are ['aa', 'ability', 'ability to', 'able', 'able to', 'about', 'about all', 'about an', 'about and', 'about any']
the shape of tfidf vector is (4986, 41401)


In [59]:
list1 = list(tfidf.idf_)               ### this is how list and idf_ works

In [60]:
list1[0]

8.415977516878012

In [61]:
print(tfidf_vector[0])

  (0, 40082)	0.23831409902091796
  (0, 40069)	0.14045952107997586
  (0, 39756)	0.17479555829616217
  (0, 39676)	0.08597027288899661
  (0, 38410)	0.25970350402276177
  (0, 37742)	0.25970350402276177
  (0, 37248)	0.25970350402276177
  (0, 37169)	0.199655852610952
  (0, 36007)	0.2196097248962045
  (0, 35684)	0.045489675095792007
  (0, 34555)	0.12619969350218824
  (0, 33577)	0.036932048841295875
  (0, 31722)	0.233557257451612
  (0, 29140)	0.22580209918210178
  (0, 29120)	0.12119141898384185
  (0, 28756)	0.11950474845365835
  (0, 27284)	0.1479996374442367
  (0, 27230)	0.08238493343280165
  (0, 26965)	0.13169020145135227
  (0, 24556)	0.09032373108648338
  (0, 23592)	0.16078839598592196
  (0, 23475)	0.048221737935928925
  (0, 22373)	0.21003884361605635
  (0, 17856)	0.1955352890172303
  (0, 17346)	0.21003884361605635
  (0, 17304)	0.08550333686272236
  (0, 15858)	0.12526249036402934
  (0, 12686)	0.0514356034401219
  (0, 12500)	0.24394024845671633
  (0, 8504)	0.15838262844934103
  (0, 5865)	0.21

In [62]:
#### word2vector

In [63]:
preprocessed_text =final["Cleaned_text"] 

In [64]:
i=0
list_of_sentence = []
for sentence in final["Cleaned_text"]:
    list_of_sentence.append(sentence.split())
w2v = Word2Vec(list_of_sentence , size= 50 , min_count = 5)
w2v_words = list(w2v.wv.vocab)

In [65]:
print(len(w2v_words))

3989


In [66]:
w2v.wv.most_similar("tasty")

[('delicious', 0.9517245292663574),
 ('crunchy', 0.9134570360183716),
 ('flavorful', 0.8984339237213135),
 ('light', 0.8863414525985718),
 ('fresh', 0.8853623867034912),
 ('fluffy', 0.8820385336875916),
 ('pretty', 0.8816474676132202),
 ('very', 0.8756937384605408),
 ('soft', 0.874908447265625),
 ('gooey', 0.8735905885696411)]

In [67]:
### average w2v 

In [68]:
total_vector = []
for sentance in list_of_sentence:
    sen_vector = np.zeros(50)
    count = 0
    for word in sentance:
        if word in w2v_words:
            count = count + 1
            sen_vector = sen_vector + w2v.wv[word]
        if count != 0:
            sen_vector = sen_vector/count
    total_vector.append(sen_vector)
            

In [69]:
len(total_vector[10])

50

In [70]:
print(total_vector[0])

[ 6.08156013e-04 -6.78834132e-03  7.09623524e-04 -7.53929354e-03
 -3.43534657e-05 -9.72903669e-04  1.27452140e-02 -3.37219166e-03
 -4.28336582e-03 -1.14290876e-02 -4.60711825e-03  7.88136305e-03
  4.40770270e-03  3.21531116e-03 -3.86610504e-03 -3.39473615e-04
  1.34545079e-03 -3.96073239e-03 -5.01245441e-03  5.11601480e-03
 -6.90710922e-03 -3.45971101e-03 -8.21304147e-04 -2.50068791e-04
  2.49394725e-03 -7.43398182e-03  9.33497015e-03  6.62442383e-03
 -1.17566965e-02 -9.38266267e-04  1.27280022e-03 -7.22660486e-03
 -1.02415070e-03 -1.51319437e-03  6.55179777e-04  5.02812712e-03
 -8.29157207e-03 -2.03944049e-03 -1.10208936e-02  8.77874020e-03
 -3.26112182e-03 -5.50128051e-03 -2.01986268e-03 -7.20346151e-04
  1.09494594e-02  2.34320317e-03  2.03369980e-03 -8.23537576e-04
 -1.08338685e-02  5.77258272e-03]


In [71]:
### tfidf_weighted_w2v

In [72]:
model = TfidfVectorizer()
model.fit(final["Cleaned_text"])
#### now we are zipping words with their idf_ value pay attention it is not tfidf value
dictionary = dict(zip(model.get_feature_names(),list(model.idf_)))

In [73]:
tfidf_weighted_w2v = []
tfidf_feat = model.get_feature_names()
for sent in tqdm(list_of_sentence):
    sentance_vector = np.zeros(50)
    tfidf_sum = 0
    for word in sent:
        if word in w2v_words and word in tfidf_feat:
            tfidf = dictionary[word]*(sent.count(word)/len(sent))
            sentance_vector += (w2v.wv[word]*tfidf)
            tfidf_sum += tfidf
        if tfidf != 0:
            sentance_vector /= tfidf_sum
    tfidf_weighted_w2v.append(sentance_vector)
            

  0%|                                                                                         | 0/4986 [00:00<?, ?it/s]<ipython-input-73-ddfe75708ec8>:12: RuntimeWarning: invalid value encountered in true_divide
  sentance_vector /= tfidf_sum
100%|██████████████████████████████████████████████████████████████████████████████| 4986/4986 [04:35<00:00, 18.09it/s]


In [74]:
len(tfidf_weighted_w2v[0])

50